# Updates

Main updates to the code one month ago


1. Make changes to the **Model Precision Notebook**(which is used to produce prediction for the entire box and save them) and **power spectrum notebook**(which is used to plot the power spectrum plot using the prediction). 

2. Add new models: model_idx = 7: Inception as classification model and R2Unet as regression model

3. Changes for run_all.s and run_c.s(new, for classification model only)

4. Add option of including velocity in **Dataset.py** and **main.py**

# Usage 

## Steps for using the two phase model

1. Run classification model with **run_c.s**, save the model to use as a mask for the second phase regression model
2. Run regression model with **run_all.s**, save the model.
3. Run **Model Precision Notebook** to produce prediction for the entire box, save them for the power specturm plot
4. Run the **Power Spectrum Notebook** to plot the power specturm

## Classification model 

### run_c.s documentation

New, mainly for running classification model

C is aconym for Classifcation, R is aconym for Regression

model_idx: 
- 0 one layer convolution C
- 2 inception C
- 3 R2Unet C



- vel: vel = 1 include velocity into input, else = 0(vel for inception don't work for now)
- loss_weight: weight for the loss function
- normalize: 1 to normalize the data, 0 not to
- conv1_out: number of hidden units for the size = 1 kernel
- conv3_out: number of hidden units for the size = 3 kernel
- conv5_out: number of hidden units for the size = 5 kernel


Sample Code is as follows, Sample output model name **pretrained/model_2@6@8@10_80_v0_n1.pth**

In [7]:
modelidx=2
target_class=0
vel=0
lossweight=80
normalize=1
conv1_out=6
conv3_out=8
conv5_out=10

modelname=$modelidx
if [ $modelidx -eq 2 ] || [ $modelidx -eq 8 ]
#if inception, add extra parameter
then
        modelname="${modelname}@${conv1_out}@${conv3_out}@${conv5_out}"
fi

name="${modelname}_${lossweight}_v${vel}_n${normalize}"
save_name="model_${name}"

python src/main.py --lr 0.001 --loss_weight $lossweight --model_idx $modelidx --epochs 30 --target_cat 'count' --target_class ${target_class} --load_model 0 \
--save_name $save_name --normalize $normalize  --record_results 0 --vel $vel > result_full_${name}.txt

SyntaxError: invalid syntax (<ipython-input-7-188c11bc9df7>, line 10)

## Regression Model

### run_all.s documentation

For the second-phase regression model

C is aconym for Classifcation, R is aconym for Regression

model_idx: 
- 4 One layer convolution + R2Unet
- 5 R2Unet + R2Unet
- 7 inception + R2Unet

- C_model: C model name used as the first-phase model
- loss_weight: weight for the loss function
- normalize: should be the same as C model
- vel: should be the same as C model




Sample Code is as follows, Sample output model name **model_7_1@model_2@6@8@10_80_v0_n1.pth** **model_7_0.6@model_2@6@8@10_80_v0_n1.pth**

In [ ]:
modelidx=7
target_class=1
# keep vel and normalize the same as the C model
vel=0
normalize=0
#remember to remove .pth
C_model="model_2@6@8@10_80_v0_n0"

for lossweight in 0.6 1
do

name="${modelidx}_${lossweight}@${C_model}"
save_name="model_${name}"

python src/main.py --lr 0.001 --loss_weight $lossweight --model_idx $modelidx --epochs 20 --target_cat 'count' --target_class ${target_class} --load_model 0 \
--save_name $save_name --normalize $normalize  --record_results 0 --vel $vel --C_model $C_model > result_full_${name}.txt

done

## Make Prediction

### model precision documentation



In the notebook, ## modify mean this cell or this line needs to be changed